In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from CCDPApy.cell_line.Fed_batch import BioProcess

pd.set_option('display.max_columns', 200)

In [2]:
from CCDPApy.file_handle import FileHandler

file = '/Users/galileo/development/Reserch/UMN_Research/CCDPA_git/CCDPApy/input_files/New_template.xlsx'
file_handler = FileHandler(file)

In [3]:
file_handler.read().keys()

dict_keys(['Measured Data', 'Feed Data', 'Separate Feed Data', 'Polynomial Degree'])

In [4]:
from CCDPApy.cell_line import bioprocess_pipeline

key_cl1 = {'use_feed_conc': True,
           'use_conc_after_feed': False,}

key_cl2 = {'use_feed_conc': False,
           'use_conc_after_feed': True,}

cell_line = bioprocess_pipeline(cell_culture_type='fed-batch',
                                input_file_name='CL1.xlsx',
                                **key_cl1)

cell_line_2 = bioprocess_pipeline(cell_culture_type='fed-batch',
                                  input_file_name='CL2.xlsx',
                                  **key_cl2)

cell_line.add_cell_line(cell_line_2)

cell_line.interactive_plot()

KeyError: "None of [Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64', name='IgG (mg/L)')] are in the [columns]"

In [ ]:
'''key_cl3 = {'use_feed_conc': False,
           'use_conc_after_feed': False,}

cell_line_3 = bioprocess_pipeline(cell_culture_type='fed-batch',
                                  input_file_name='CL3.xlsx',
                                  **key_cl3)
cell_line_3.disp_cell_lines()'''

In [ ]:
import pandas as pd
from CCDPApy.helper.helper import input_path, split_df

# Read all sheets into a dictionary
file_path = input_path('CL2.xlsx')
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Access each sheet's DataFrame
for sheet_name, df in sheets_dict.items():
    print("Sheet:", sheet_name)
    print(df[['Experiment Information']].head(2))  # Print the first few rows of each sheet's DataFrame

In [ ]:
data = sheets_dict['Sheet1']
bp = BioProcess(data=data, use_feed_conc=False, use_conc_after_feed=True)
bp.in_process()

In [ ]:
a = bp.get_conc_df()
a.head()

In [ ]:
b = bp.get_cumulative_conc_df()
b.head()

In [ ]:
c = bp.get_sp_rate_df()
c

In [ ]:
bp.sp_rate

In [ ]:
spc = bp.get_species('nh3')
spc.cumulative_conc

In [ ]:
spc.sp_rate

In [ ]:
cell = bp.get_species('cell')
cell.growth_rate

In [ ]:
igg = bp.get_species('product')
igg.sp_rate

In [ ]:
glt = bp.get_species('glutamine')
glt.cumulative_conc

In [ ]:
from CCDPApy.helper.helper import split_df
TARGET_COLUMNS = [
    'Experiment Information', 
    'Experimental Data', 
    'Cell',
    'Oxygen',
    'Product',
    'Separate Feed Added', 
    'Concentration Before Feeding',
    'Concentration After Feeding', 
    'Feed Concentration', 
    'Calculated Cumulative Concentration',
    ]
df_list = split_df(sheets_dict['3'], TARGET_COLUMNS)

In [ ]:
exp_info_df = df_list[0]
exp_data_df = df_list[1]
cell_df = df_list[2]
oxygen_df = df_list[3]
product_df = df_list[4]
separate_feed_df = df_list[5]
conc_before_feed_df = df_list[6]
conc_after_feed_df = df_list[7]
feed_conc_df = df_list[8]
measured_cumulative_conc_df = df_list[9]

In [ ]:
DATE_KEY = 'Date (MM/DD/YY)'
TIME_KEY = 'Time (HH:MM:SS AM/PM)'
RUN_TIME_D_KEY = 'Run Time (day)'
RUN_TIME_H_KEY = 'Run Time (hr)'

df = exp_data_df
date = df[DATE_KEY]
time = df[TIME_KEY]
run_time_day = df[RUN_TIME_D_KEY]
run_time_hour = df[RUN_TIME_H_KEY]

In [ ]:
if run_time_day.isnull().all() and run_time_hour.isnull().all():
    if pd.api.types.is_datetime64_any_dtype(date):
        date = date.apply(lambda x: x.date())
    
    if pd.api.types.is_datetime64_any_dtype(time):
        time = time.apply(lambda x: x.time())

    if (date==time).all():
        date_time = pd.to_datetime(date.astype(str))
    else:
        date_time = pd.to_datetime(date.astype(str) + ' ' + time.astype(str))
    time_diff = date_time - date_time.iat[0]
    run_time_hour = time_diff.dt.total_seconds() / 3600.0
    run_time_day = time_diff.dt.days

    df[RUN_TIME_D_KEY] = run_time_day
    df[RUN_TIME_H_KEY] = run_time_hour

In [ ]:
from CCDPApy.helper import create_col_indices

create_col_indices(exp_data_df)

In [ ]:
product_df.columns

In [ ]:
exp_data_df